In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch import flatten

In [ ]:
idLookUpTable = pd.read_csv("idLookUpTable.csv")
train_set = pd.read_csv("training.csv")
test_set = pd.read_csv("test.csv")

In [ ]:
train_set.isnull().sum()

In [ ]:
for idx,image in enumerate(train_set.Image):
    train_set["Image"][idx] = np.fromstring(image, dtype=int, sep=" ").reshape(96,96)

    
for idx,image in enumerate(test_set.Image):
    test_set["Image"][idx] = np.fromstring(image, dtype=int, sep=" ").reshape(96,96)

In [ ]:
plt.imshow(train_set.iloc[0].Image, cmap="gray")

In [ ]:
train_set.dropna(subset=["left_eye_center_x","right_eye_center_x","mouth_center_bottom_lip_x"], inplace=True)
#train_set.dropna(axis=1, inplace=True)

In [ ]:
#missing value imputation

train_set_notNull = train_set.dropna(axis=0, inplace=False)

left_eye_inner_corner_x_diff = (train_set_notNull.left_eye_inner_corner_x - train_set_notNull.left_eye_center_x).mean()
left_eye_inner_corner_y_diff = (train_set_notNull.left_eye_inner_corner_y - train_set_notNull.left_eye_center_y).mean()
right_eye_inner_corner_x_diff = (train_set_notNull.right_eye_inner_corner_x - train_set_notNull.right_eye_center_x).mean()
right_eye_inner_corner_y_diff = (train_set_notNull.right_eye_inner_corner_y - train_set_notNull.right_eye_center_y).mean()

left_eye_outer_corner_x_diff = (train_set_notNull.left_eye_outer_corner_x - train_set_notNull.left_eye_center_x).mean()
left_eye_outer_corner_y_diff = (train_set_notNull.left_eye_outer_corner_y - train_set_notNull.left_eye_center_y).mean()
right_eye_outer_corner_x_diff = (train_set_notNull.right_eye_outer_corner_x - train_set_notNull.right_eye_center_x).mean()
right_eye_outer_corner_y_diff = (train_set_notNull.right_eye_outer_corner_y - train_set_notNull.right_eye_center_y).mean()

left_eyebrow_inner_end_x_diff = (train_set_notNull.left_eyebrow_inner_end_x - train_set_notNull.left_eye_center_x).mean()
left_eyebrow_inner_end_y_diff = (train_set_notNull.left_eyebrow_inner_end_y - train_set_notNull.left_eye_center_y).mean()
right_eyebrow_inner_end_x_diff = (train_set_notNull.right_eyebrow_inner_end_x - train_set_notNull.right_eye_center_x).mean()
right_eyebrow_inner_end_y_diff = (train_set_notNull.right_eyebrow_inner_end_y - train_set_notNull.right_eye_center_y).mean()

left_eyebrow_outer_end_x_diff = (train_set_notNull.left_eyebrow_outer_end_x - train_set_notNull.left_eye_center_x).mean()
left_eyebrow_outer_end_y_diff = (train_set_notNull.left_eyebrow_outer_end_y - train_set_notNull.left_eye_center_y).mean()
right_eyebrow_outer_end_x_diff = (train_set_notNull.right_eyebrow_outer_end_x - train_set_notNull.right_eye_center_x).mean()
right_eyebrow_outer_end_y_diff = (train_set_notNull.right_eyebrow_outer_end_y - train_set_notNull.right_eye_center_y).mean()

mouth_left_corner_x_diff = (train_set_notNull.mouth_left_corner_x - train_set_notNull.mouth_center_bottom_lip_x).mean()
mouth_left_corner_y_diff = (train_set_notNull.mouth_left_corner_y - train_set_notNull.mouth_center_bottom_lip_y).mean()

mouth_right_corner_x_diff = (train_set_notNull.mouth_right_corner_x - train_set_notNull.mouth_center_bottom_lip_x).mean()
mouth_right_corner_y_diff = (train_set_notNull.mouth_right_corner_y - train_set_notNull.mouth_center_bottom_lip_y).mean()

mouth_center_top_lip_x_diff = (train_set_notNull.mouth_center_top_lip_x - train_set_notNull.mouth_center_bottom_lip_x).mean()
mouth_center_top_lip_y_diff = (train_set_notNull.mouth_center_top_lip_y - train_set_notNull.mouth_center_bottom_lip_y).mean() + 2


In [ ]:
train_set["left_eye_inner_corner_x"].fillna(train_set.left_eye_center_x + left_eye_inner_corner_x_diff, inplace=True)
train_set["left_eye_inner_corner_y"].fillna(train_set.left_eye_center_y + left_eye_inner_corner_y_diff, inplace=True)
train_set["left_eye_outer_corner_x"].fillna(train_set.left_eye_center_x + left_eye_outer_corner_x_diff, inplace=True)
train_set["left_eye_outer_corner_y"].fillna(train_set.left_eye_center_y + left_eye_outer_corner_y_diff, inplace=True)

train_set["right_eye_inner_corner_x"].fillna(train_set.right_eye_center_x + right_eye_inner_corner_x_diff, inplace=True)
train_set["right_eye_inner_corner_y"].fillna(train_set.right_eye_center_y + right_eye_inner_corner_y_diff, inplace=True)
train_set["right_eye_outer_corner_x"].fillna(train_set.right_eye_center_x + right_eye_outer_corner_x_diff, inplace=True)
train_set["right_eye_outer_corner_y"].fillna(train_set.right_eye_center_y + right_eye_outer_corner_y_diff, inplace=True)

train_set["left_eyebrow_inner_end_x"].fillna(train_set.left_eye_center_x + left_eyebrow_inner_end_x_diff, inplace=True)
train_set["left_eyebrow_inner_end_y"].fillna(train_set.left_eye_center_y + left_eyebrow_inner_end_y_diff, inplace=True)
train_set["left_eyebrow_outer_end_x"].fillna(train_set.left_eye_center_x + left_eyebrow_outer_end_x_diff, inplace=True)
train_set["left_eyebrow_outer_end_y"].fillna(train_set.left_eye_center_y + left_eyebrow_outer_end_y_diff, inplace=True)

train_set["right_eyebrow_inner_end_x"].fillna(train_set.right_eye_center_x + right_eyebrow_inner_end_x_diff, inplace=True)
train_set["right_eyebrow_inner_end_y"].fillna(train_set.right_eye_center_y + right_eyebrow_inner_end_y_diff, inplace=True)
train_set["right_eyebrow_outer_end_x"].fillna(train_set.right_eye_center_x + right_eyebrow_outer_end_x_diff, inplace=True)
train_set["right_eyebrow_outer_end_y"].fillna(train_set.right_eye_center_y + right_eyebrow_outer_end_y_diff, inplace=True)

train_set["mouth_left_corner_x"].fillna(train_set.mouth_center_bottom_lip_x + mouth_left_corner_x_diff, inplace=True)
train_set["mouth_left_corner_y"].fillna(train_set.mouth_center_bottom_lip_y + mouth_left_corner_y_diff, inplace=True)

train_set["mouth_right_corner_x"].fillna(train_set.mouth_center_bottom_lip_x + mouth_right_corner_x_diff, inplace=True)
train_set["mouth_right_corner_y"].fillna(train_set.mouth_center_bottom_lip_y + mouth_right_corner_y_diff, inplace=True)

train_set["mouth_center_top_lip_x"].fillna(train_set.mouth_center_bottom_lip_x + mouth_center_top_lip_x_diff, inplace=True)
train_set["mouth_center_top_lip_y"].fillna(train_set.mouth_center_bottom_lip_y + mouth_center_top_lip_y_diff, inplace=True)


In [ ]:
#train_set, val_set = np.split(train_set,[int(.9*len(train_set))])

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
def plotKeypoints(data):
    plt.imshow(data.Image, cmap="gray")
    plt.scatter(data[:-1:2], data[1:-1:2])

In [ ]:
plotKeypoints(train_set.iloc[6943])

In [ ]:
class ToTensor(object):
    """Convert ndarrays in sample to Tensors."""

    def __call__(self, sample):
        image, landmarks = sample['image'], sample['keypoints']
        return {'image': torch.from_numpy(image),
                'keypoints': torch.tensor(landmarks)}

In [ ]:
class faceLandmarksDataset(Dataset):
    def __init__(self, data ,transform=ToTensor()):
        self.data = data
        self.transform = transform
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self,idx):
        keypoints = self.data.iloc[idx,:-1]
        image = self.data.iloc[idx].Image
        
        sample = {'image': image, 'keypoints': keypoints}
        
        return self.transform(sample)

In [ ]:
class FaceKeypointDetectionModel(nn.Module):
    def __init__(self):
        super(FaceKeypointDetectionModel, self).__init__()
        self.conv1 = nn.Conv2d(1,32, kernel_size=3)
        self.conv2 = nn.Conv2d(32,64, kernel_size=3)
        self.conv3 = nn.Conv2d(64,96, kernel_size=3)
        self.conv4 = nn.Conv2d(96,128, kernel_size=3)
        self.conv5 = nn.Conv2d(128,256, kernel_size=3)
        
        self.fc1 = nn.Linear(9216, 1024)
        self.fc2 = nn.Linear(1024,128)
        self.fc3 = nn.Linear(128, 30)
        
        self.BN1 = nn.BatchNorm2d(32)
        self.BN2 = nn.BatchNorm2d(64)
        self.BN3 = nn.BatchNorm2d(96)
        self.BN4 = nn.BatchNorm2d(128)
        self.BN5 = nn.BatchNorm2d(256)
        
        self.pool = nn.AvgPool2d(2,2)
        #self.pool = nn.MaxPool2d(2,2)
        self.dropout = nn.Dropout(p=0.1)
        
    def forward(self, x):
        
        x = F.relu(self.conv1(x))
        x = self.BN1(x)
        x = self.pool(x)

        x = F.relu(self.conv2(x))
        x = self.BN2(x)
        x = self.dropout(x)
        x = self.pool(x)
        
        x = F.relu(self.conv3(x)) 
        x = self.BN3(x)
        x = self.dropout(x)
        x = self.pool(x)
        
        x = F.relu(self.conv4(x)) 
        x = self.BN4(x)
        x = self.dropout(x)
        
        
        x = F.relu(self.conv5(x)) 
        x = self.BN5(x)
        x = self.dropout(x)
        
        
        x = flatten(x, start_dim=1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        
        return x

In [ ]:
def train(network, epochs, optimizer, criterion, train_loader, val_loader = None):
    
    for epoch in range(epochs):
        network.train()
        epoch_loss = 0
        for batch_idx, data in enumerate(train_loader):
            images, keyPoints = data["image"].float().to(device) , data["keypoints"].float().to(device)
            images = images.unsqueeze(1)
            optimizer.zero_grad()
            output = network(images)
            loss = criterion(output, keyPoints)
            epoch_loss += loss.item()
            loss.backward()
            optimizer.step()
        
        epoch_loss /= len(train_loader)
        print('Train Epoch: {} \tTrain Loss: {:.6f}'.format(epoch, epoch_loss))

        if ( val_loader is not None ):
            network.eval()
            epoch_loss = 0.0

            for batch_idx, data in enumerate(val_loader):
                images, keyPoints = data["image"].float().to(device) , data["keypoints"].float().to(device)
                images = images.unsqueeze(1)
                optimizer.zero_grad()
                output = network(images)
                loss = criterion(output, keyPoints)
                epoch_loss += loss.item()

            epoch_loss /= len(val_loader)
            print('Train Epoch: {} \tValidation Loss: {:.6f}'.format(epoch, epoch_loss))


In [ ]:
trainDataset = faceLandmarksDataset(train_set)
#valDataset = faceLandmarksDataset(val_set)

In [ ]:
trainLoader = DataLoader(trainDataset, batch_size = 16)
#valLoader = DataLoader(valDataset, batch_size = 16)

In [ ]:
model = FaceKeypointDetectionModel().to(device)

In [ ]:
lr = 0.0001
optimizer = optim.Adam(model.parameters(), lr=lr)
criterion = nn.MSELoss()

In [ ]:
train(model.to(device), 1, optimizer, criterion, trainLoader)

In [ ]:
#torch.save(model.state_dict(), <pre_trained_model_path>)

In [ ]:
#pre_trained_model_path = "/content/drive/MyDrive/facial_keypoint_detection/model_final")
#model.load_state_dict(torch.load(model_path))

In [ ]:
test_image_id = 716

test_image = torch.tensor(test_set.iloc[test_image_id].Image)
plt.imshow(test_image, cmap="gray")
pred = model(torch.tensor(test_image).unsqueeze(0).unsqueeze(1).float().to(device))
pred = torch.tensor(pred.squeeze().to(device))
plt.scatter(pred[::2].to("cpu"), pred[1::2].to("cpu"))

In [ ]:
test_images = torch.tensor(test_set.Image).reshape(-1,1,96,96).float().to(device)

In [ ]:
with torch.no_grad():
    pred = model(test_images[0].reshape(-1,1,96,96))
    for image in test_images[1:]:
        pred = torch.cat((pred,model(image.reshape(-1,1,96,96))),0)
    
    pred = abs(pred)

In [ ]:
lookid_list = list(lookid_data["FeatureName"])
imageID = list(lookid_data["ImageId"]-1)
pred_list = list(pred)

In [ ]:
rowid = lookid_data['RowId']
rowid=list(rowid)

In [ ]:
feature = []
for f in list(lookid_data['FeatureName']):
    feature.append(lookid_list.index(f))

In [ ]:
preded = []
for x,y in zip(imageID,feature):
    preded.append(float(pred_list[x][y]))

In [ ]:
rowid = pd.Series(rowid,name = 'RowId')

In [ ]:
loc = pd.Series(preded,name = 'Location')

In [ ]:
submission = pd.concat([rowid,loc],axis = 1)

In [ ]:
submission.to_csv('/content/drive/MyDrive/facial_keypoint_detection/face_key_detection_submission.csv',index = False)

In [ ]:
test_image_id = 122

plt.imshow(test_set.iloc[test_image_id].Image, cmap="gray")
plt.scatter(pred[test_image_id][::2].to("cpu"), pred[test_image_id][1::2].to("cpu"))